In [5]:
!pip install pydub ffmpeg librosa

                                              0.0/253.7 kB ? eta -:--:--
     ------------------------------------- 253.7/253.7 kB 16.2 MB/s eta 0:00:00
                                              0.0/2.6 MB ? eta -:--:--
     ------------------                       1.2/2.6 MB 37.4 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 33.7 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 28.1 MB/s eta 0:00:00
                                              0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 1.0/1.0 MB 31.2 MB/s eta 0:00:00
                                              0.0/63.0 kB ? eta -:--:--
     ---------------------------------------- 63.0/63.0 kB ? eta 0:00:00
                                              0.0/184.7 kB ? eta -:--:--
     ---------------------------------------- 184.7/184.7 kB ? eta 0:00:00
                                              0.0/75.2 kB ? eta -:--:--
     -------------------


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
from pydub import AudioSegment
import librosa

In [39]:
!ffmpeg -version 

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
def calculate_bpm(file_path):
    """
    Calculate the BPM of an audio file using librosa.

    Args:
    file_path (str): Path to the audio file.

    Returns:
    float: Estimated BPM of the audio.
    """
    y, sr = librosa.load(file_path, sr=None)

    onset_env = librosa.onset.onset_strength(y, sr=sr)

    # Estimate the tempo
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)

    return tempo

In [24]:
def change_bpm(audio, original_bpm, target_bpm):
    """
    Changes the BPM of an audio file by adjusting the speed.
    This will also change the pitch of the audio.

    Args:
    audio (AudioSegment): The audio segment to adjust.
    original_bpm (float): The original BPM of the audio.
    target_bpm (float): The target BPM to achieve.

    Returns:
    AudioSegment: The modified audio segment with new BPM.
    """
    ratio = target_bpm / original_bpm

    new_audio = audio.speedup(playback_speed=ratio)

    return new_audio

In [34]:
def transition_audio(file1_path, file2_path, decibel_reduction):
    """
    Reduces the volume of the last 5 seconds of file1 and overlaps file2 over the last 5 seconds.
    
    Args:
    file1_path (str): Path to the first mp3 file.
    file2_path (str): Path to the second mp3 file.
    decibel_reduction (int): Number of decibels to reduce in the last 5 seconds of the first file.

    Returns:
    transition (AudioSegment): The combined audio segment.
    """
    audio1 = AudioSegment.from_file(file1_path, "mp3")
    audio2 = AudioSegment.from_file(file2_path, "mp3")

    bpm_audio1 = calculate_bpm(file1_path)
    bpm_audio2 = calculate_bpm(file2_path)

    # Change the BPM of the slower audio file to match the faster audio file
    if bpm_audio1 > bpm_audio2:
        audio2 = change_bpm(audio2, bpm_audio2, bpm_audio1)
    elif bpm_audio1 < bpm_audio2:
        audio1 = change_bpm(audio1, bpm_audio1, bpm_audio2)

    # Reduce the volume of the last 5 seconds of the first audio file
    last_5_seconds = audio1[-5000:].apply_gain(-decibel_reduction)
    first_part = audio1[:-5000]

    # Combine the first part with the modified last 5 seconds
    modified_audio1 = first_part + last_5_seconds

    # Overlay the start of the second file over the last 5 seconds of the modified first file
    transition = modified_audio1.overlay(audio2, position=-5000)

    return transition

In [38]:
audio = transition_audio("inspiration.mp3", "realta", 10)


c:\Users\arora\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified